# Hedonic Pricing

We often try to predict the price of an asset from its observable characteristics. This is generally called **hedonic pricing**: How do the unit's characteristics determine its market price?

In the lab folder, there are three options: housing prices in pierce_county_house_sales.csv, car prices in cars_hw.csv, and airbnb rental prices in airbnb_hw.csv. If you know of another suitable dataset, please feel free to use that one.

1. Clean the data and perform some EDA and visualization to get to know the data set.
2. Transform your variables --- particularly categorical ones --- for use in your regression analysis.
3. Implement an ~80/~20 train-test split. Put the test data aside.
4. Build some simple linear models that include no transformations or interactions. Fit them, and determine their RMSE and $R^2$ on the both the training and test sets. Which of your models does the best?
5. Make partial correlation plots for each of the numeric variables in your model. Do you notice any significant non-linearities?
6. Include transformations and interactions of your variables, and build a more complex model that reflects your ideas about how the features of the asset determine its value. Determine its RMSE and $R^2$ on the training and test sets. How does the more complex model your build compare to the simpler ones?
7. Summarize your results from 1 to 6. Have you learned anything about overfitting and underfitting, or model selection?
8. If you have time, use the sklearn.linear_model.Lasso to regularize your model and select the most predictive features. Which does it select? What are the RMSE and $R^2$? We'll cover the Lasso later in detail in class.

In [9]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression

#1
df = pd.read_csv('linearModels/lab/data/airbnb_hw.csv')
# drop rows with NaN values
df = df.dropna()
# Remove '$' and ',' then convert to float
df.loc[:, 'Price'] = df['Price'].replace('[\$,]', '', regex=True).astype(float)

#2
# Selecting relevant numeric and categorical variables
numeric_columns = ['Host Id','Review Scores Rating (bin)','Zipcode','Beds','Number of Records','Number Of Reviews','Review Scores Rating']
categorical_columns = ['Neighbourhood ','Property Type', 'Room Type']

y = df['Price']
X = df[numeric_columns + categorical_columns].copy()
# Maxmin normalization function; Sci-kit calls it the "standard scaler" (changed for constant values)
def maxmin(z):
    return (z - z.min()) / (z.max() - z.min()) if z.min() != z.max() else z

X[numeric_columns] = X[numeric_columns].apply(maxmin) # Normalize X

# One-hot encode categorical variables
X = pd.get_dummies(X, columns=categorical_columns, drop_first=True)
#3
# Split the sample:
X_train, X_test, y_train, y_test = train_test_split(X, y, # Feature and target variables
                                                    test_size=0.2, # Split the sample 80 train/ 20 test
                                                    random_state=65) # For replication purposes

#4
train_reg = LinearRegression(fit_intercept=False).fit(X_train, y_train) # Fit the linear model
y_train_pred = train_reg.predict(X_train)
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
print(f"Train RMSE: {train_rmse}")
print(f"Train R squared measure:{train_reg.score(X, y)}") # R squared measure

test_reg = LinearRegression(fit_intercept=False).fit(X_test, y_test) # Fit the linear model
y_test_pred = test_reg.predict(X_test)
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
print(f"Test RMSE: {test_rmse}")
print(f"Test R squared measure:{test_reg.score(X, y)}") # R squared measure

#The test values did better, as the RMSE is lower. This means there is less of an error. The difference in the R squared values are negligible.


Train RMSE: 133.2070273711007
Train R squared measure:0.26160889247882957
Test RMSE: 102.98672612727043
Test R squared measure:0.25292558891740324
